In [ ]:
import spacy
import pandas as pd

import unidecode 


#from spacy.lang.nl import STOP_WORDS
from string import punctuation
from collections import Counter


In [ ]:
import transformers
undisputed_best_model = transformers.MBartForConditionalGeneration.from_pretrained(
    "ml6team/mbart-large-cc25-cnn-dailymail-nl-finetune"
)
tokenizer = transformers.MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")
summarization_pipeline = transformers.pipeline(
    task="summarization",
    model=undisputed_best_model,
    tokenizer=tokenizer,
)
summarization_pipeline.model.config.decoder_start_token_id = tokenizer.lang_code_to_id[
    "nl_XX"
]



In [ ]:
undisputed_best_model.to("cuda")

In [ ]:
df=pd.read_csv("data_03.csv",sep="|")

In [ ]:
import re
def reducing_incorrect_character_repeatation(text):

    # Pattern matching for all case alphabets
    Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)
    
    # Limiting all the  repeatation to two characters.
    Formatted_text = Pattern_alpha.sub(r"\1\1", text) 
    
    # Pattern matching for all the punctuations that can occur
    Pattern_Punct = re.compile(r'([.,/#!$%^&*?;:{}=_`~()+-])\1{1,}')
    
    # Limiting punctuations in previously formatted string to only one.
    Combined_Formatted = Pattern_Punct.sub(r'\1', Formatted_text)
    
    # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
    Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)
    return Final_Formatted

In [ ]:
num=444

In [ ]:
l=df['nl_body']
import re
import string

documents_clean = []
for d in l:
    d=re.sub(r'(?<=[.,;,:])(?=[^\s])', r' ', d)
    


    document_test= unidecode.unidecode(d)
    document_test = document_test.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', document_test)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    document_test = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    document_test = document_test.lower()
    document_test=reducing_incorrect_character_repeatation(document_test)

    document_test = re.sub(r"[^a-zA-Z0-9:$-,%.?!]+", ' ', document_test) 
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    document_test = re.sub(r"[^a-zA-Z:$-,%.?!]+", ' ', document_test)

    documents_clean.append(document_test)

In [ ]:
l=df['nl_body']



In [ ]:
summary=[]
qw=0
for a in l:
    

    t= str(qw) +"|"+ summarization_pipeline(
        a,
        do_sample=True,
        top_p=0.75,
        top_k=50,
        num_beams=4,
        min_length=50,
        early_stopping=True,
        truncation=True,
    )[0]["summary_text"]

    with open('summ.csv', 'a',encoding="utf-8-sig") as f:
    
        f.writelines(t)
        f.write('\n')

    qw=qw+1
    print(qw)

(783,)

In [ ]:
df2=pd.DataFrame(summary)
df2.to_csv("translate00-7xx.csv",encoding="utf-8-sig")

In [14]:
df3=pd.read_csv('summ.csv',sep="|")


In [17]:
df3

,ind,nl_summ
0,0,Demissionair minister van de Brusselse Hoofdst...
1,1,De hierna volgende tekst is de Duitse vertalin...
2,2,Het Koninklijk Huis van Utrecht heeft een wijz...
3,3,De geleidelijke transformatie van de bestaande...
4,4,Het ministerie van Financiën heeft een nieuw m...
...,...,...
778,778,De hierna volgende tekst is de Duitse vertalin...
779,779,Het koninklijk besluit tot wijziging van het k...
780,780,Het Europees Parlement en de Raad van 7 oktobe...
781,781,Het koninklijk besluit tot verlenging van de m...


In [18]:
nlsumm=df3['nl_summ']

In [19]:
df['nl_sum']=nlsumm

In [25]:
df.to_csv("data05.csv",encoding="utf-8-sig",sep="|")

In [ ]:
df.iloc[num]['nl_body']

In [ ]:
nl_tax_keywords = ["Aanslagjaar","successierechten","Wetboek der Successierechten","fiscale hervorming","fiscale gunstregime","belasting over de toegevoegde waarde",
        "BTW","kadastraal inkomen","onroerende voorheffing","Algemene Administratie van de Fiscaliteit","Inkomstenbelastingen","belastingvermindering","fiscale stimulans",
        "belastbaar tijdperk","Belcotax-on-web","Bedrijfsvoorheffing","Het Wetboek van de inkomstenbelastingen","het Wetboek diverse rechten en taksen",
        "fiscale schuldvorderingen","het Wetboek diverse rechten en taksen","fiscale schuldvorderingen","belastingen op beroepsinkomsten","vennootschapsbelastingen",
        "belastingen op onroerende goederen","verbruiksbelastingen","verkeersbelastingen","afvalbelastingen","provinciebelastingen","milieutaksen","belastingaangifte",
        "belastingteruggave","Biztax","Intervat","Rv-on-web","FinProf","MyMinFinPro","roerende voorheffing","vennootschapsbelasting","rechtspersonenbelasting","belastingkorting", 
        "Bronbelasting", "fiscale bepalingenbelastingontduiking","Belastingontwijking","fiscaal voordeel","Aftrekbare besteding","Belastingvrije som", "Belastingkrediet",
        "Decumul","Forfaitaire beroepskosten","Gemeenschappelijke aanslag","Gemiddelde aanslagvoet","Huwelijksquotiënt","Inkomstenjaar","Marginale aanslagvoet",
        "Onroerende voorheffing","Opcentiemen","Progressieve belastingtarieven","Verbeterd gemiddeld tarief","Vrijstelling met progressievoorbehoud"]

In [ ]:
for a  in nl_tax_keywords:
    for q  in tokenizer.tokenize(a):
        print (q.pos_)
    



    

In [36]:
data=pd.read_csv('data06.csv',sep="|")

In [37]:
data

,Date,Title,Numac,Link FR,Link NL,nl_body,fr_body,nl_sum
0,14/01/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,BRUSSELS HOOFDSTEDELIJK GEWEST 8 JANUARI 2020....,REGION DE BRUXELLES-CAPITALE 8 JANVIER 2020. -...,Demissionair minister van de Brusselse Hoofdst...
1,16/01/2020,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE ...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,FEDERALE OVERHEIDSDIENST BINNENLANDSE ZAKEN 10...,SERVICE PUBLIC FEDERAL INTERIEUR 10 DECEMBRE 2...,De hierna volgende tekst is de Duitse vertalin...
2,16/01/2020,SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 20...,2.020040e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,FEDERALE OVERHEIDSDIENST FINANCIEN 7 DECEMBER ...,SERVICE PUBLIC FEDERAL FINANCES 7 DECEMBRE 201...,Het Koninklijk Huis van Utrecht heeft een wijz...
3,24/01/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,MINISTERIE VAN DE FRANSE GEMEENSCHAP 20 DECEMB...,MINISTERE DE LA COMMUNAUTE FRANÇAISE 20 DECEMB...,De geleidelijke transformatie van de bestaande...
4,28/01/2020,SERVICE PUBLIC FEDERAL FINANCES\nAdministratio...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,FEDERALE OVERHEIDSDIENST FINANCIEN Algemene A...,SERVICE PUBLIC FEDERAL FINANCES Administratio...,Het ministerie van Financiën heeft een nieuw m...
...,...,...,...,...,...,...,...,...
778,04/04/2022,SERVICE PUBLIC FEDERAL FINANCES\n29 MARS 2021....,2.022021e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,FEDERALE OVERHEIDSDIENST FINANCIEN 29 MAART 2...,SERVICE PUBLIC FEDERAL FINANCES 29 MARS 2021....,De hierna volgende tekst is de Duitse vertalin...
779,04/04/2022,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n23 MA...,2.022031e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,FEDERALE OVERHEIDSDIENST SOCIALE ZEKERHEID 2...,SERVICE PUBLIC FEDERAL SECURITE SOCIALE 23 M...,Het koninklijk besluit tot wijziging van het k...
780,04/04/2022,SERVICE PUBLIC FEDERAL FINANCES\n23 FEVRIER 20...,2.022040e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,FEDERALE OVERHEIDSDIENST FINANCIEN 23 FEBRUAR...,SERVICE PUBLIC FEDERAL FINANCES 23 FEVRIER 20...,Het Europees Parlement en de Raad van 7 oktobe...
781,04/04/2022,"SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONC...",2.022202e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,"FEDERALE OVERHEIDSDIENST WERKGELEGENHEID, ARBE...","SERVICE PUBLIC FEDERAL EMPLOI, TRAVAIL ET CONC...",Het koninklijk besluit tot verlenging van de m...


In [34]:
data.drop(['Unnamed: 0'],axis=1,inplace=True)

In [35]:
data.to_csv("data06.csv",encoding="utf-8-sig",sep="|",index=False)